<a href="https://colab.research.google.com/github/DeepCodeSec/ml1000-p1/blob/working_models/Project1_wine_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working code for wine classification problem.

**Dataset:**
* wine quality data retrieved from https://archive.ics.uci.edu/ml/datasets/Wine+Quality

* using the white wine csv only (had more observations than the red)


**Problem:**
* how do we test if the wine is high quality and should be priced accordingly?